In [1]:
import os
import openai
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Insert your OpenAI API Key: ")

Insert your OpenAI API Key: ········


O primeiro passo é importar o PDF com as informações necessárias

In [8]:
from langchain.document_loaders import PyPDFLoader

pdf_path = "data/domCasmurro.pdf"
loader = PyPDFLoader(file_path=pdf_path)
document = loader.load()

Vamos deixar a possibilidade do desenvolvedor adicionar informações complementares a vector store

In [9]:
from langchain.document_loaders import TextLoader
loader = TextLoader("data/domCasmurro-adicionais.txt")
complementary_document = loader.load()

Após isso, vamos dividir o texto em chunks

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=50, 
    separators=["\n\n", "\n", " ", ""]
)
texts = text_splitter.split_documents(document+complementary_document)

Vamos escolher um embedder para transformar as sentenças em vetores. Aqui, em vez de utilizar es embeddings da OpenAi, vou escolher utilizar um embedder da HuggingFace para reduzir os custos

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")

Agora, vamos criar o banco de dados vetorial e persistir

In [12]:
from langchain.vectorstores import FAISS

# -- demora ----------------------------
vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local("data/dom-casmurro-huggingface.db")

Em usos gerais, quando formos utilizar a vector store, devemos instancia o mesmo modelo de embedding e importar a vector store do arquivo local

In [13]:
%%time
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")
vector_store = FAISS.load_local("data/dom-casmurro-huggingface.db", embeddings)

CPU times: user 1.64 s, sys: 453 ms, total: 2.09 s
Wall time: 1.73 s


## RetrievalQA

Nas vezes em que você não quer uma conversação, mas assim, apenas uma resposta, deve-se usar o `RetrievalQA`

In [14]:
from langchain.chains  import RetrievalQA
from langchain.llms import OpenAI

# Escolha o modelo de lingaugem
llm = OpenAI(model='text-davinci-003', temperature=0)

# Isntancie o retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

In [15]:
qa.run("Quem são os personagens principais de dom Casmurro?")

' Os personagens principais de Dom Casmurro são Bento Santiago, Capitu, Escobar, Sancha, José Dias e Glória.'

In [16]:
qa.run("Qual a relação entre Bento e Capitu?")

' Bento e Capitu são amantes.'

In [17]:
qa.run("Em que cidade se passa a história?")

' A história de "Dom Casmurro" se passa na cidade do Rio de Janeiro, no Brasil.'

In [31]:
qa.run("Quem é Ezequiel?")

' Ezequiel é o filho de Capitu e o narrador do livro.'

## ConversationalRetrievalChain

Agora, caso exista a necessidade de uma conversação, dai devemos instanciar outra cadeia: o ConversationalRetrievalChain. Nesse caso, temos uma nova ferramenta que se encarrega de guardar a memória das mensagens

In [18]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Com o buffer de memória, ai sim podemos instanciar o `ConversationalRetrievalChain`

In [19]:
from langchain.chains import ConversationalRetrievalChain

llm = OpenAI(temperature=0)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vector_store.as_retriever(),
    memory=memory
)

In [20]:
query = "Quem são os personagens principais de dom Casmurro?"
result = qa({"question": query})
result["answer"]

' Os personagens principais de Dom Casmurro são Bento Santiago, Capitu, Escobar, Sancha, José Dias e Glória.'

In [21]:
query = "Quais desses formam um casal?"
result = qa({"question": query})
result["answer"]

' Escobar e Sancha formam um casal.'

In [22]:
query = "E alem desses dois?"
result = qa({"question": query})
result["answer"]

' Escobar e Maria.'

In [23]:
query = "Tem certeza?"
result = qa({"question": query})
result["answer"]

' Não, não temos certeza de que Escobar e Maria formam um casal. Não há nenhuma menção a eles formando um casal no contexto.'

In [24]:
query = "Mas e dentre os personagens principais, com quem Bento se envolve?"
result = qa({"question": query})
result["answer"]

' Bento Santiago se envolve com Capitu, a quem ele se casa.'

In [25]:
query = "E ela tem um filho?"
result = qa({"question": query})
result["answer"]

' Sim, Capitu tem um filho.'

In [27]:
query = "E quem é o pai do filho de Capitu?"
result = qa({"question": query})
result["answer"]

' O pai do filho de Capitu é Mano Cosme.'

In [29]:
query = "E qual é o nome do filho de Capitu?"
result = qa({"question": query})
result["answer"]

' O nome do filho de Capitu não é mencionado.'

In [32]:
query = "E quem é Ezequiel?"
result = qa({"question": query})
result["answer"]

' Ezequiel é o filho de Capitu e o narrador do livro.'